In [56]:
import pandas as pd

In [57]:
selectCrime2 = pd.read_csv ("selectCrime2.csv")
selectCrime2

,Unnamed: 0,geometry,cartodb_id,crimeType,count,nearestSVPoint,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice,All
0,0,POINT (-75.220592 39.91443),14,Burglary Non-Residential,1,2967,0.038,0.000,0.166,0.005,0.005,0.000,0.174,0.487,0.004,0.0,0.285
1,1,POINT (-75.220592 39.91443),15,Aggravated Assault No Firearm,1,2967,0.038,0.000,0.166,0.005,0.005,0.000,0.174,0.487,0.004,0.0,0.013
2,2,POINT (-75.220592 39.91443),23,Robbery Firearm,1,2967,0.038,0.000,0.166,0.005,0.005,0.000,0.174,0.487,0.004,0.0,0.656
3,3,POINT (-75.06662799999999 40.049264),29,Thefts,1,8024,0.012,0.002,0.092,0.061,0.085,0.004,0.120,0.447,0.028,0.0,0.769
4,4,POINT (-75.161446 39.962334),53,Thefts,1,10759,0.031,0.006,0.219,0.003,0.047,0.006,0.235,0.303,0.069,0.0,0.769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171445,174718,POINT (-75.150063 39.916778),2600417,Motor Vehicle Theft,1,3720,0.085,0.001,0.097,0.032,0.088,0.026,0.338,0.221,0.024,0.0,0.454
171446,174719,POINT (-75.075478 40.063138),2600421,Motor Vehicle Theft,1,10676,0.028,0.000,0.089,0.008,0.139,0.010,0.341,0.262,0.055,0.0,0.454
171447,174720,POINT (-75.17222599999999 39.997811),2589417,Motor Vehicle Theft,1,804,0.027,0.005,0.313,0.044,0.071,0.025,0.069,0.284,0.036,0.0,0.454
171448,174721,POINT (-75.151239 39.991324),2589424,Motor Vehicle Theft,1,9774,0.039,0.006,0.240,0.008,0.158,0.048,0.224,0.189,0.013,0.0,0.454


In [58]:
selectCrime2 = selectCrime2.drop(columns=['Unnamed: 0','All','geometry','nearestSVPoint'])

Vars=["wall","lives","building","infrastructure","road","sidewalk","sky","green","transportation","publicservice"]
selectCrime2[Vars] = selectCrime2[Vars].apply(lambda x: round((x-x.min())/(x.max()-x.min()),2))

selectCrime2 = pd.melt(selectCrime2, id_vars=['cartodb_id','crimeType','count'])
selectCrime2 = selectCrime2.groupby(["variable","value"]).sum("count")
selectCrime2 = selectCrime2.reset_index()
selectCrime2.head()

,variable,value,cartodb_id,count
0,building,0.00,58301664,54
1,building,0.01,503256668,386
2,building,0.02,722526307,608
3,building,0.03,1431903458,1119
4,building,0.04,1589717144,1369


In [59]:
selectCrime2.dtypes

variable       object
value         float64
cartodb_id      int64
count           int64
dtype: object

In [83]:
selection3 = selectCrime2.loc[(selectCrime2["variable"]=="green")]

chart=alt.Chart(selection3).mark_point(
).encode(x='value:Q',y='count:Q')

chart + chart.transform_regression('value','count').mark_line()

alt.LayerChart(...)

In [81]:
import altair as alt
from vega_datasets import data
alt.data_transformers.enable('json')

alt.Chart(selectCrime2).mark_point(
).encode(x='value:Q',y='count:Q',color="variable:N"
).facet(
    facet='variable:N',
    columns=2
).resolve_scale(
    y='independent'
).interactive()

alt.FacetChart(...)

In [61]:
source

,yield,variety,year,site
0,27.00000,Manchuria,1931,University Farm
1,48.86667,Manchuria,1931,Waseca
2,27.43334,Manchuria,1931,Morris
3,39.93333,Manchuria,1931,Crookston
4,32.96667,Manchuria,1931,Grand Rapids
...,...,...,...,...
115,58.16667,Wisconsin No. 38,1932,Waseca
116,47.16667,Wisconsin No. 38,1932,Morris
117,35.90000,Wisconsin No. 38,1932,Crookston
118,20.66667,Wisconsin No. 38,1932,Grand Rapids


In [62]:
import altair as alt
from vega_datasets import data

source = data.barley()

error_bars = alt.Chart(source).mark_errorbar(extent='ci').encode(
  x=alt.X('yield:Q', scale=alt.Scale(zero=False)),
  y=alt.Y('variety:N')
)

points = alt.Chart(source).mark_point(filled=True, color='black').encode(
  x=alt.X('yield:Q', aggregate='mean'),
  y=alt.Y('variety:N'),
)

error_bars + points

alt.LayerChart(...)